In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fftfreq, fft
from scipy.signal.windows import blackman
%matplotlib tk

In [2]:
V_SF = pd.read_csv('MotorVRB_2025.csv')
V_2F = pd.read_csv('MotorV2F_2025.csv')

M_SF = pd.read_csv('MotorRB_2025.csv')
M_2F = pd.read_csv('MotorRR2F_2025.csv')

In [5]:
skew_SF, kurt_SF = V_SF.skew(), V_SF.kurt()
skew_2F, kurt_2F = V_2F.skew(), V_2F.kurt()

In [7]:
kurt_SF

Sinal-C0             0.260538
Sinal-C0-Filter      0.440064
Sinal-C20            0.068755
Sinal-C20-Filter     0.053468
Sinal-C40            0.514144
Sinal-C40-Filter     0.559837
Sinal-C60            0.845667
Sinal-C60-Filter     0.870544
Sinal-C80            1.019537
Sinal-C80-Filter     1.108986
Sinal-C100           0.927918
Sinal-C100-Filter    1.042745
dtype: float64

In [6]:
skew_SF

Sinal-C0            -0.441823
Sinal-C0-Filter      0.019113
Sinal-C20           -0.171415
Sinal-C20-Filter    -0.067517
Sinal-C40           -0.423383
Sinal-C40-Filter    -0.107421
Sinal-C60           -0.544471
Sinal-C60-Filter    -0.152315
Sinal-C80           -0.569591
Sinal-C80-Filter    -0.183189
Sinal-C100          -0.558188
Sinal-C100-Filter   -0.192523
dtype: float64

In [8]:
kurt_2F

Sinal-C0            -0.143895
Sinal-C0-Filter     -0.359090
Sinal-C20            0.162123
Sinal-C20-Filter     0.380827
Sinal-C40            0.102423
Sinal-C40-Filter     0.312136
Sinal-C60            2.555527
Sinal-C60-Filter     1.902245
Sinal-C80            1.321530
Sinal-C80-Filter     0.860659
Sinal-C100           0.841091
Sinal-C100-Filter    0.767024
dtype: float64

In [9]:
skew_2F

Sinal-C0            -0.493394
Sinal-C0-Filter     -0.236020
Sinal-C20           -0.605719
Sinal-C20-Filter    -0.373364
Sinal-C40           -0.622033
Sinal-C40-Filter    -0.410696
Sinal-C60           -1.421237
Sinal-C60-Filter    -0.779184
Sinal-C80           -1.092486
Sinal-C80-Filter    -0.580080
Sinal-C100          -0.910523
Sinal-C100-Filter   -0.455603
dtype: float64

In [4]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize = (12,8))
ax1.plot(V_SF['Sinal-C80-Filter'], linewidth=0.5)

ax2.plot(M_2F['Sinal-C80-Filter'], linewidth=0.5)

In [5]:
def fft_np(signal, sample_rate = 50000):
    fft_np = np.fft.fft(signal)
    fft_freq = np.fft.fftfreq(signal.shape[0], 1/sample_rate)
    return fft_np, fft_freq

In [7]:
signal = M_2F['Sinal-C0'].to_numpy()
N = len(signal)
fft_np, fft_freq = fft_np(signal)
plt.figure(figsize = (12,8))
plt.plot(fft_freq[:N // 2], np.abs(fft_np[:N // 2]), linewidth = 0.5)

In [14]:
def fft_scipy(signal, fs=50000):
    N = len(signal)
    T = 1.0 / fs

    # Aplicar ventana Blackman
    w = np.blackman(N)
    signal_windowed = signal * w

    # FFT
    fft_sc = fft(signal_windowed)
    fft_freq = fftfreq(N, T)[:N//2]

    return fft_sc, fft_freq, w

In [23]:
signal = M_SF['Sinal-C0'].to_numpy()
N = len(signal)
fft_sc, fft_freq, window = fft_scipy(signal)

# FFT de la ventana
fft_window = fft(window)
window_amplitude = np.abs(fft_window[:N//2])
window_amplitude /= np.max(window_amplitude)  # Normalizar a 1

# FFT de la señal normalizada
scaling_factor = np.sum(window) / N
signal_amplitude = (1.0 / N) * np.abs(fft_sc[:N//2]) / scaling_factor

# Evitar ceros para log
signal_amplitude[signal_amplitude <= 1e-20] = 1e-20

# Graficar
plt.figure(figsize=(14, 8))
plt.semilogy(fft_freq[1:], signal_amplitude[1:], linewidth=0.7)
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Amplitud')
plt.tight_layout()
plt.show()